In [1]:
import os
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from tqdm import tqdm
import concurrent.futures
import json

/Users/hchaves/Github/IMT/fake-landscape-detector/ai/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

In [3]:
RAW_DATA_PATH = "../data/img/raw"
RAW_IMAGE_FILENAMES = os.listdir(RAW_DATA_PATH)
TEXT_CONDITION = "a photography of"

In [4]:
# image_captions = {}

# for raw_image_filename in tqdm(RAW_IMAGE_FILENAMES):
#     raw_image_path = os.path.join(RAW_DATA_PATH, raw_image_filename)
#     raw_image = Image.open(raw_image_path).convert("RGB")
#     inputs = processor(raw_image, TEXT_CONDITION, return_tensors="pt")
#     out = model.generate(**inputs)
#     caption = processor.decode(out[0], skip_special_tokens=True)
#     image_captions[raw_image_filename] = caption

In [5]:
def get_caption(raw_image_filename):
    raw_image_path = os.path.join(RAW_DATA_PATH, raw_image_filename)
    raw_image = Image.open(raw_image_path).convert("RGB")
    inputs = processor(raw_image, TEXT_CONDITION, return_tensors="pt")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return raw_image_filename, caption

In [10]:
image_captions = {}
NUM_THREADS = 5

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    # Start the load operations and mark each future with its URL
    future_to_filename = {executor.submit(get_caption, filename): filename for filename in RAW_IMAGE_FILENAMES[:100]}
    for future in tqdm(concurrent.futures.as_completed(future_to_filename), total=len(future_to_filename), desc='Processing Images'):
        filename = future_to_filename[future]
        try:
            image_captions[filename] = future.result()[1]
        except Exception as exc:
            print(f'An exception occurred while processing {filename}: {exc}')

Processing Images:   0%|          | 0/100 [00:00<?, ?it/s]/Users/hchaves/Github/IMT/fake-landscape-detector/ai/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Processing Images: 100%|██████████| 100/100 [01:10<00:00,  1.43it/s]


In [11]:
CAPTIONS_DATA_DIR = "../data/json"
CAPTIONS_FILENAME = "image_captions_sample_100.json"
CAPTIONS_PATH = os.path.join(CAPTIONS_DATA_DIR, CAPTIONS_FILENAME)

with open(CAPTIONS_PATH, 'w') as f:
    json.dump(image_captions, f)